In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy import stats

from one.api import ONE
from brainbox.io.one import SessionLoader

import sys
sys.path.append('/home/crombie/code/ibl_photometry/src')
from iblphotometry.loaders import KceniaLoader

from iblnm.io import LOCAL_CACHE
from iblnm.util import EVENT2COLOR
from iblnm.resp import get_responses

In [2]:
one = ONE(cache_dir=LOCAL_CACHE)

/home/crombie/mnt/ccu-iblserver/datasets.pqt does not appear to be a valid table. Skipping


In [3]:
df_sessions = pd.read_parquet('metadata/sessions.pqt').query('local_photometry == True')

In [4]:
df_sessions = df_sessions[df_sessions['target'].apply(len) == 1]
df_sessions['target_NM'] = df_sessions.apply(lambda x: f"{x['target'][0]}-{x['NM']}", axis='columns')
df_sessions['target_NM'].value_counts()

target_NM
DR-5HT     617
LC-NE      432
VTA-DA     308
SNc-DA     279
NBM-ACh    132
MR-5HT      53
Name: count, dtype: int64

In [6]:
# Pick a target
target_NM = 'NBM-ACh'
df_target = df_sessions.query('target_NM == @target_NM')
df_target.groupby(['subject', 'session_type']).count()['session_n']

subject    session_type
ZFM-06946  training        11
ZFM-06948  biased          11
           training        27
ZFM-08751  habituation      3
           training        26
ZFM-08757  habituation      3
           training        25
ZFM-08813  habituation      3
           training        10
ZFM-08814  habituation      3
           training        10
Name: session_n, dtype: int64

In [169]:
## NBM mouse, no photometry data extracted !!

# subject = 'ZFM-06946'
# session = df_sessions.query('subject == @subject').iloc[-1]
# eid = session['eid']
# session

In [54]:
subject = 'ZFM-06948'
session_type = 'biased'
session_n = 0
session = df_sessions.query('(subject == @subject) & (session_type == @session_type)').iloc[session_n]
eid = session['eid']
session[['subject', 'NM', 'target', 'session_type', 'session_n', 'start_time', 'eid', 'local_photometry']]

subject                                        ZFM-06948
NM                                                   ACh
target                                             [NBM]
session_type                                      biased
session_n                                             28
start_time                    2024-01-12T11:52:32.158403
eid                 580d1208-d910-4fc9-8c28-8364c92dbd46
local_photometry                                    True
Name: 2266, dtype: object

In [55]:
loader = KceniaLoader(one)
photometry = loader.load_photometry_data(eid)
gcamp = photometry['raw_calcium']
iso = photometry['raw_isosbestic']
print(gcamp.columns)

Index(['Region5G'], dtype='object')


In [56]:
# roi = 'Region6G'
roi = gcamp.columns[0]

In [57]:
# trials_fpath = one.eid2path(eid) / 'alf' / 'task_00' / '_ibl_trials.table.pqt'
trials_fpath = one.eid2path(eid) / 'alf' / '_ibl_trials.table.pqt'
df_trials = pd.read_parquet(trials_fpath)

FileNotFoundError: [Errno 2] No such file or directory: '/home/crombie/mnt/ccu-iblserver/mainenlab/Subjects/ZFM-06948/2024-01-12/001/alf/_ibl_trials.table.pqt'

> /home/crombie/miniconda3/envs/iblenv/lib/python3.10/site-packages/pandas/io/common.py(882)get_handle()
    880         else:
    881             # Binary mode
--> 882             handle = open(handle, ioargs.mode)
    883         handles.append(handle)
    884 



ipdb>  c


In [ ]:
buffer = 15
timings = [col for col in df_trials.columns if col.endswith('_times')]
t0 = df_trials[timings].min().min()
t1 = df_trials[timings].max().max()
gcamp = gcamp.iloc[gcamp.index.searchsorted(t0 - buffer):gcamp.index.searchsorted(t1 + buffer)].copy()
iso = iso.iloc[iso.index.searchsorted(t0 - buffer):iso.index.searchsorted(t1 + buffer)].copy()

In [21]:
def _load_event_times(session, df_trials):
    session['cue_times'] = df_trials['goCue_times'].values
    session['movement_times'] = df_trials['firstMovement_times'].values
    session['reward_times'] = df_trials.query('feedbackType == 1')['feedback_times'].values
    session['omission_times'] = df_trials.query('feedbackType == -1')['feedback_times'].values
    return session

session = _load_event_times(session, df_trials)

events = ['cue', 'movement', 'reward', 'omission']
psths = []
for event in events:
    responses, tpts = get_responses(gcamp[roi], session[f'{event}_times'])
    psths.append(responses)

In [22]:
%matplotlib tk

fig = plt.figure(figsize=(12, 8))
grid = gridspec.GridSpec(2, 3)

ax2 = fig.add_subplot(grid[1, 0])
ax3 = fig.add_subplot(grid[1, 1])
ax4 = fig.add_subplot(grid[1, 2])
ax1 = fig.add_subplot(grid[0, :])

ax1.plot(gcamp[roi])
ax1.plot(iso[roi], color='gray')
ax1.set_ylabel('Signal (a.u.)')
ax1.set_title(f"{session['subject']}, {session['session_type']}, session {session['session_n']}")

colors = [EVENT2COLOR[event] for event in events]
for event, responses, color, ax in zip(events, psths, colors, [ax2, ax3, ax4, ax4]):
    ax.plot(tpts, responses.mean(axis=0), color=color, label=event)
    ax.plot(tpts, responses.mean(axis=0) - stats.sem(responses, axis=0), ls='--', color=color)
    ax.plot(tpts, responses.mean(axis=0) + stats.sem(responses, axis=0), ls='--', color=color)
    ax.axvline(0, ls='--', color='black', alpha=0.5)
    ax.axhline(0, ls='--', color='gray', alpha=0.5)
    ax.set_xlabel('Time (s)')
    if event != 'omission':
        ax.set_title(event.capitalize())
    ax.ticklabel_format(axis='y', style='sci', scilimits=[-2, 2])